In [9]:
%matplotlib inline
import lyricsgenius as lg
import numpy as np
import random
import requests as rq
import sys
import io
from bs4 import BeautifulSoup
from keras.callbacks import LambdaCallback, ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, LSTM, Activation, Flatten, Dropout, Embedding, TimeDistributed, Input, InputLayer
from keras.optimizers import RMSprop
from collections import Counter
from datetime import datetime
import keras
import keras.callbacks
from keras.callbacks import TensorBoard
from keras.utils import np_utils
import pandas as pd
from keras.applications import VGG16
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [10]:
def makeLyricsText(artistName):
    title = artistName.replace(" ", "") + ".txt"
    file = open(title, "w")
    genius = lg.Genius('EVEbdlvJhu4oRGFz56kQIrETSGLVaJDvHkwbNzsyu1ysjU0Jc8x0w641ZqdfXmc8', skip_non_songs=True, 
                   excluded_terms=["(Remix)", "(Live)"], remove_section_headers=True)
    artists = [artistName]
    
    def getLyrics(arr, k):
        c = 0
        for name in arr:
            try:
                songs = (genius.search_artist(name, max_songs=k, sort='popularity')).songs
                s = [song.lyrics for song in songs]
                file.write("\n \n    \n \n".join(s))
                c += 1
                print(f"Songs grabbed:{len(s)}")
            except:
                print(f"some exception at {name}: {c}")
            
    getLyrics(artists, 20)
    file = open(title, encoding = "UTF-8")
    specificLyrics = file.read()
    specificLyrics = specificLyrics.lower()
    file.close()
    print(specificLyrics)
    return specificLyrics

In [11]:
def createModel(lyrics):
    chars = sorted(list(set(lyrics)))
    char_indices = dict((c, i) for i, c in enumerate(chars))
    indices_char = dict((i, c) for i, c in enumerate(chars))

    # finding num. of characters in text & number of unique characters
    n_characters = len(lyrics)
    n_vocabulary = len(chars)

    print("Total characters in lyric set: " , n_characters)
    print("Total vocabulary in lyric set: " , n_vocabulary)

    seqlen = 40
    step = seqlen
    sentences = []
    for i in range(0, len(lyrics) - seqlen - 1, step):
        sentences.append(lyrics[i: i + seqlen + 1])
    x = np.zeros((len(sentences), seqlen, len(chars)), dtype=np.bool)
    y = np.zeros((len(sentences), seqlen, len(chars)), dtype=np.bool)
    for i, sentence in enumerate(sentences):
        for t, (char_in, char_out) in enumerate(zip(sentence[:-1], sentence[1:])):
            x[i, t, char_indices[char_in]] = 1
            y[i, t, char_indices[char_out]] = 1
            
    model = Sequential()
    model.add(LSTM(128, input_shape=(seqlen, len(chars)), return_sequences=True))
    model.add(Dense(len(chars), activation='softmax'))

    model.compile(
        loss = 'categorical_crossentropy',
        optimizer=RMSprop(learning_rate=0.01), 
        metrics=['categorical_crossentropy', 'accuracy'])

    def sample(preds, temperature=1.0):
        """Helper function to sample an index from a probability array."""
        preds = np.asarray(preds).astype('float64')
        preds = np.exp(np.log(preds) / temperature)  # softmax
        preds = preds / np.sum(preds)                #
        probas = np.random.multinomial(1, preds, 1)  # sample index
        return np.argmax(probas)    

    def on_epoch_end(epoch, _):
        """Function invoked at end of each epoch. Prints generated text."""
        print()
        print('----- Generating text after Epoch: %d' % epoch)

        start_index = random.randint(0, len(lyrics) - seqlen - 1)


        for diversity in [0.2, 0.5, 1.0]:
            print('----- diversity:', diversity)

            generated = ''
            sentence = lyrics[start_index: start_index + seqlen]
            generated += sentence
            print('----- Generating with seed: "' + sentence + '"')
            sys.stdout.write(generated)

            for i in range(400):
                x_pred = np.zeros((1, seqlen, len(chars)))
                for t, char in enumerate(sentence):
                    x_pred[0, t, char_indices[char]] = 1.
                preds = model.predict(x_pred, verbose=0)
                next_index = sample(preds[0, -1], diversity)
                next_char = indices_char[next_index]

                sentence = sentence[1:] + next_char

                sys.stdout.write(next_char)
                sys.stdout.flush()
            print()

    print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

    model.fit(x, y,
              batch_size=128,
              epochs=50,
              callbacks=[print_callback])
    
    return model

In [12]:
artist1model = createModel(makeLyricsText("Taylor Swift"))

Searching for songs by Taylor Swift...

Song 1: "​cardigan"
Song 2: "​exile"
Song 3: "Lover"
Song 4: "​the 1"
Song 5: "Look What You Made Me Do"
Song 6: "​betty"
Song 7: "End Game"
Song 8: "ME!"
Song 9: "You Need To Calm Down"
Song 10: "​august"
Song 11: "...Ready for It?"
Song 12: "Blank Space"
Song 13: "​my tears ricochet"
Song 14: "Delicate"
Song 15: "​invisible string"
Song 16: "Call It What You Want"
"Bad Blood (Remix)" is not valid. Skipping.
Song 17: "​the last great american dynasty"
"I Don’t Wanna Live Forever" is not valid. Skipping.
Song 18: "Cruel Summer"
Song 19: "Style"
Song 20: "​seven"

Reached user-specified song limit (20).
Done. Found 20 songs.
Songs grabbed:20
vintage tee, brand new phone
high heels on cobblestones
when you are young, they assume you know nothing
sequin smile, black lipstick
sensual politics
when you are young, they assume you know nothing

but i knew you
dancin' in your levi's
drunk under a streetlight, i
i knew you
hand under my sweatshirt
baby, k

Epoch 1/50
8/9 [=========================>....] - ETA: 0s - loss: 3.4428 - categorical_crossentropy: 3.4428 - accuracy: 0.1332
----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "ver, ah
take me out, and take me home
yo"
ver, ah
take me out, and take me home
yo    t    t      e  t    ttt   t e  t      t t      t          t   tt    t t t     t to    t t       t     t     t t   t t  e       t t  o           t      t    t tt   t   t o           t   t t    t  e   t            t  t    t   d      e o  e  t    t    t     t t   t         tt  t  t  tt t  ttt     tt o       o       tt           ttt t  t   e t    t  o t       t  t         t       t t    t  o t t  
----- diversity: 0.5
----- Generating with seed: "ver, ah
take me out, and take me home
yo"
ver, ah
take me out, and take me home
yo  to ratt t e n ouao ot  d otttdetoe  tdoe e b tto er t   w    et  t   tuto e t n te c o i o  rit o d   t it ttwt yot t   het at ttwttot   nn    o t t et t c o d trrt   ee

i got some ine iou iou wal  in  in war it in  in ii i i at in ine iou ine ine iou war  ine ial wan  ine iou ie in ie ine iou ii ar ine iou iou war iou iou i an  an  in  in  in ii ine it it ian ia i in iat iou wan ware i a  in war war ile in i in, iou ii i the i i i i in iour ine iou i in it in in ie i in i an ine ine ial ine iou ine iin i it in wat in i in ia  in you ine ias ii ii iou iat ine iou ii  in  i
----- diversity: 0.5
----- Generating with seed: "ah
and you heard about me, ooh
i got som"
ah
and you heard about me, ooh
i got some ire you waneeeoun i in me it it, youle you wiat is r ao

ou jhan in e i wis ite mau iia wall iou anl ine i an iou  is  inee you e i i ve iu i k you it iou wou  iul in i bau ihe you ) in you it iis iike iou aiu you iik ine iou ahit in whan thet iik lou you ial is it e washe line iout ine iou you we ine iou ite wan  il  ine iou ioouee ine tou  il in su ine iil shio aai iit you il our in aarr ial a
----- diversity: 1.0
----- Generating with seed: "ah
and 

end tohe of me meded she stou whon the lot the my me fook sher shet me
loow
ind you dook you me the coou for the whow shet look me she
look whet me meve shet stam 
i keon book the lide whot m nock known the myou noooke whan the know mone you kink you knome you me dook whot whou the you mever the knole lo
----- diversity: 1.0
----- Generating with seed: "lie
you said the gun was mine
isn't cool"
lie
you said the gun was mine
isn't cool on  ot 't wnonke fhe
gathises
s?
focate yon'
yourlttilke inl of liokt
lokn what in maor wethe wall dyome coou
in?  ihall
whow yow- heas thet nld whome hhet youhasse yowr liol, youlkest me

hovew me ha t sh whs then soof ghakes you do ghomnnck youlloknng, [oh anstmyscack seec

o takenkg mcsz, in thethas gosll
knee's she nave what you've mesn shet thone
sird bothe sithoin thyoy kiikewy th fe
call g
9/9 [==============================] - 38s 4s/step - loss: 2.2214 - categorical_crossentropy: 2.2214 - accuracy: 0.3571
Epoch 10/50
8/9 [========================

out r at what the a know a know, it haht ooh-oh, i ah what, what rfat you iy what you, bal, it that you weth that waat what whou just yoch, what yoae the what fooo foow
oh what liou (can that yoohmh, saali fan hea howh the haal it what just you'st it  
9/9 [==============================] - 38s 4s/step - loss: 1.9896 - categorical_crossentropy: 1.9896 - accuracy: 0.4195
Epoch 14/50
8/9 [=========================>....] - ETA: 0s - loss: 1.9704 - categorical_crossentropy: 1.9704 - accuracy: 0.4275
----- Generating text after Epoch: 13
----- diversity: 0.2
----- Generating with seed: "en
i had a marvelous time ruining everyt"
en
i had a marvelous time ruining everything whe se mand whe pall what you wan the wan a dan whe like what you wan the wan the wan the wan the wan the wan the wann the whan you wan a ma do now in whe and wan a do wann the wan whe wan the wan the wan the wan whe wand i wan the wan the wann the wan the wan the wand i wan the wan a wan the wan the wan the wan the mise t

...

like can you just nother like
[cheruth ther like
[cheruth ther like
[cher ther like
[cher ther like
[cheruthing there)
and i wanna be your first string ther like
[chorus: brendon uries there)
and i wanna be your first string there)
and i wanna be your first string ther like
[cheruth ther like
[cherust fromise there)
and i can the like of ther like
[cheruth ther like
[cher ther like
[chorus: brendon uries there)
and 
----- diversity: 0.5
----- Generating with seed: "stop (can you stop?)
like can you just n"
stop (can you stop?)
like can you just nat ther fill there there)
and i wanna be your firstames what they like wher ther mistlike at there)
(never like
you never there on ther like
[cher ther like
[vernether like
[cheruth ther like
[cher there there's there)
and therelike
and that call it what they like
a baby ther like
[h-oh, oh-oh, oh-oh, oh-oh, oh-oh, oh-oh, oh-oh, oh-oh, oh-oh
(oh
it's at, shice, theres all that ther like
(heard all
----- diversity: 1.0
----- Generating with seed

the one of mame of it all
and it what you want, yeah
call it what you just made me do
look what you we, you just made me do
look what you made me do
look what you want you just made me do
look what you just made me do
look what you just made me do
look what you just made me do
look what you want you just made m
----- diversity: 1.0
----- Generating with seed: "... delicate?

third floor on the west s"
... delicate?

third floor on the west she elmer do cayl do fum)
i'vengomen it would've roe winlo hooh
illome"she vefored youn my?w stowty pque you soed ot it just me, you we?h oh, oh-oh (e
)
)

thir would've so ho foo to filco

you nodg your hone, bun
lomever, the bagt
summer lwok oh-oh, oh-your caniea for me to
werld you wast me
lover
and you know thming lighcall this call
’t at you would've me do
flow your whome you sunt mesboh
when 
9/9 [==============================] - 38s 4s/step - loss: 1.2015 - categorical_crossentropy: 1.2015 - accuracy: 0.6439
Epoch 36/50
8/9 [=================

boby, is every and arme, iilleted so
i gann than carmed
igh"
you still seepulded
are, us and you knipg my?w?
you know that?
is oild, dee, do los, things cale me lover go wut ale me
i grees that yeverd a
9/9 [==============================] - 39s 4s/step - loss: 1.0904 - categorical_crossentropy: 1.0904 - accuracy: 0.6774
Epoch 40/50
8/9 [=========================>....] - ETA: 0s - loss: 1.0655 - categorical_crossentropy: 1.0655 - accuracy: 0.6847
----- Generating text after Epoch: 39
----- diversity: 0.2
----- Generating with seed: "osé flowing with your chosen family
and "
osé flowing with your chosen family
and it's gonna be forever
it's a think i've seen the back of to to the can to me on the stop things we can and mady something (i hon't tell to canter my like
and wanna be
your e-e-e-eead to cant mest to me, wo
look what you just made me do
look what you made me do
look what you just made me do
look what you just made me do
look what you just made me do
look what you made me do
loo

i'll be mike a mistlertire, we never go out of style
you wast me
and i'll tell toll of the seen donnther things we don't was seen you wast me
and we true
this in the middle of in the peace
if i like you wast me
and i'll tall to me-e-e

so then the me-to calm down, where rult of me
and i creel sementeen can i go for my myshong

you were me-exm
worst this you i'll got the 
----- diversity: 0.5
----- Generating with seed: "car"
and then canceled my plans just in "
car"
and then canceled my plans just in call temmes relding in the made me worst time-eers nobody time, we're me hem
worst this price you know i'm insain
"
crys yer what you wast me
and we never gonould me, we't stepl truth me
i'm in the scan to me were
you know a love to come
'cause ie what you wast never of in me (

prick is go call when whis well you wast me...
we trow style, when you wast me... we true
i gut on your carmed me worle 
----- diversity: 1.0
----- Generating with seed: "car"
and then canceled my plans just in "
c

was this fill that i dad you love thing that i sas you love thing that i ever deaild take me out, and teke thas i paid
that i ever deaving to make a lot of you
and i promind to read your parkind inder all that to this to me on the players
and you love sting
and it too soo

oh you know i love to so long from your g
----- diversity: 1.0
----- Generating with seed: "
baby, that's the fun of you
and i promi"

baby, that's the fun of you
and i promise that you're come aid that they was the plase fleent pacts, crome, w
"wey, yeah, whoai tow thos?
goysulike flassame for me
to keyp, ar
it's atougm

thie treis that hife be shending you never made me do
"
 t  
 
 ee p leave you loven thing thai my
suel, thai't like a fight)
to staps at beawtrelide the hope oug the ing)
i'm shink i did ligfs know
now using ty nver good ary ind gons
we aloun the en
9/9 [==============================] - 39s 4s/step - loss: 0.9037 - categorical_crossentropy: 0.9037 - accuracy: 0.7350
Epoch 49/50
8/9 [==============

In [13]:
artist2model = createModel(makeLyricsText("Bob Dylan"))

Searching for songs by Bob Dylan...

Song 1: "Murder Most Foul"
Song 2: "Blowin’ in the Wind"
Song 3: "The Times They Are A-Changin’"
Song 4: "All Along the Watchtower"
Song 5: "Like a Rolling Stone"
Song 6: "Make You Feel My Love"
Song 7: "Hurricane"
Song 8: "Don’t Think Twice, It’s All Right"
Song 9: "Mr. Tambourine Man"
Song 10: "A Hard Rain’s A-Gonna Fall"
Song 11: "Subterranean Homesick Blues"
Song 12: "Desolation Row"
Song 13: "Tangled Up in Blue"
Song 14: "Masters of War"
Song 15: "Knockin’ on Heaven’s Door"
Song 16: "Tempest"
Song 17: "It’s Alright, Ma (I’m Only Bleeding)"
Song 18: "Girl from the North Country"
Song 19: "I Contain Multitudes"
Song 20: "Ballad of a Thin Man"

Reached user-specified song limit (20).
Done. Found 20 songs.
Songs grabbed:20
'twas a dark day in dallas, november '63
a day that will live on in infamy
president kennedy was a-ridin' high
good day to be livin' and a good day to die
being led to the slaughter like a sacrificial lamb
he said, "wait a minute

Epoch 1/50
 9/10 [==========================>...] - ETA: 0s - loss: 3.3437 - categorical_crossentropy: 3.3437 - accuracy: 0.1214
----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: " something for the birdman of alcatraz
p"
 something for the birdman of alcatraz
p       e                      e  o t     e      o      n                              o           e    o                d
e        e  t     d  a  n   s               n       t     i  re t t   o    a         e            o e t e            t o        o                   t           t t                             t     e  n             t  o   a o        s                    t    e t                
----- diversity: 0.5
----- Generating with seed: " something for the birdman of alcatraz
p"
 something for the birdman of alcatraz
p titos nie nea h   nt iu  it o  stf no  rststn ol o   hne iowthl e u  
n  e hle m  iea s n t 
  e  l t  l a eee h a th 
i   t a a  t  n  banh     nlia t w   t o    tl ot

stand the the the doull the the the the the doullllllllllllllllllllllllllllllly the the d the the the the the the the the the the the doud the the the the the dond the the the the dound sou d the the the the the the doullllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllling the the the dound the the doully ind the the the d the the dot the the the d the the the the the the 
----- diversity: 0.5
----- Generating with seed: "rty going on behind the grassy knoll
sta"
rty going on behind the grassy knoll
stand they yout tout s thand wast allind therd thand kall nd tee t anctang
the dout toutsren the thangind sland the s lly they theallllly comelllat wind yond tle whelmand
st the yond ine
tat the d the the de the d coulelld bute thel s s thet tout you tonk nothend the cland
ang teey soutt the thelllllt yobd yound the yound soust youlllayothe me tou dound the the mel che coun thery pley
plllly bu d the
----- diversity: 1.0
----- Generating with seed: "rty going on 

your brothers are comin', the sele the whe the the the wat the no he the oney ur the co whe whe play whe the the "the shat the whe sall it fe the an the he pooe whe lathe walling an blat war the the "the the ple yor of the the the the wime the pray whe prat whe balle the the the the ling i' he beathing the play hall the the sto sto the the lathe the munge the seo lo the the whe dere the the ghime lo bo the ghe in che paps a
----- diversity: 1.0
----- Generating with seed: "s on the way
your brothers are comin', t"
s on the way
your brothers are comin', the mit mine
a cir the ello wheenang hhe the b ap mets of hin the, in der juthud be rodive yow share, ceamine
hho beed she fomint he o the che uno he rap he , beuthe mio hou youpceathe whe haighe io she it the a roi't therere brears
the  he ou t 'l fouthing mee toe hiu yout th w
 fererd “l “sle bu seo whe whir then monicas hint you maning boo the the suep
you she plarsand ahe ho tee mime he deartrg
10/10 [==============================] 

 ormund ant how goothhes illey
id hive dod lome out foo tole the makn, but thi
wee whst tow a dur over's gove mave soove wollath to sum broke
litely to  teeve mindsann
blaice


a nowler goils hes that caw "they ulsars
tho chen come you
iprey iisdat the wide love’l werd hooss
hem lown widd nekpe?

and the crazk
on the get my beatting as the stan he
10/10 [==============================] - 38s 4s/step - loss: 2.0406 - categorical_crossentropy: 2.0406 - accuracy: 0.3926
Epoch 14/50
 9/10 [==========================>...] - ETA: 0s - loss: 1.9932 - categorical_crossentropy: 1.9932 - accuracy: 0.4057
----- Generating text after Epoch: 13
----- diversity: 0.2
----- Generating with seed: "
now he looked so immaculately frightful"

now he looked so immaculately frightfuld the stand the rast the sand the sand the play "the stand the stand and the wand the play "whe wast to the stand the sand the wall the pand the ward the know the stand the pand the sand the stand the play "the wall the ward in 

...

your song on the gooners who was was sail the something is happening
i has like it all that the sone fight of the sing to the bast
the wite had with the road

and the side were and the sould the times him of the right of the road

and the wite was no one to the times in the wind
we to the times around the meat more follow

and it a parker like it any are for the loveling on the gooners the side of the age
----- diversity: 0.5
----- Generating with seed: "icize what you can't understand
your son"
icize what you can't understand
your song on my wind to be over the singin' on the wind
wet unled the falles the past roing for some with the wind to the jaster came the was like it all that me un the stand in the some with spenking in the windle walled on the wind

be and pringsed might pack of the time under to me in the roath with your live

and the sing the stand with wat roaring that no one the witely was no geater
where the wook w
----- diversity: 1.0
----- Generating with seed: "icize wh

was was play a song for me
i'm goons of comen, frueter wolld of my say
a d stand or a freentience
body that someare for me
is blue

you wast dist to stand what did you gonnasside breath, what's not slick on the sacrices
and sonet lonely, shind someone bect
for mace and i freeting
play bast like body that life on your moo
----- diversity: 1.0
----- Generating with seed: "e to one who lives there
for she once wa"
e to one who lives there
for she once was rearou’d inkid, “to this goint a can plooks whas are thing to soning
treaden's dan whilanguch for sey
nevbrandin' spitn ournen me all i’m a know dait’s a-rone?"
z“dishe wat enwh wealays and sone amered arn the world from his vis bark what white moet asa is do what shout and to blownew go monn, blospentir who wasted loubin' brofn colly broundly
 o go llacedsoon
daille ne wase sone put slippin's p
10/10 [==============================] - 39s 4s/step - loss: 1.3313 - categorical_crossentropy: 1.3313 - accuracy: 0.5919
Epoch 36/50
 9/10 [===

there looked counters
d. ows collop and
fr. tay "prat lictich, when’s just with more ucry with the doorquls, or play it for i frovil
beft sacked up in betann drame
play it for the highways frumledred upsid
hade
woncams,, while so eo
10/10 [==============================] - 39s 4s/step - loss: 1.2402 - categorical_crossentropy: 1.2402 - accuracy: 0.6195
Epoch 40/50
 9/10 [==========================>...] - ETA: 0s - loss: 1.2228 - categorical_crossentropy: 1.2228 - accuracy: 0.6267
----- Generating text after Epoch: 39
----- diversity: 0.2
----- Generating with seed: "ectric violin
on desolation row

dr. fil"
ectric violin
on desolation row

dr. files come follow
and in the parter it feel?
the really betts the rood for a wask to was can see well

while the right was but now it me world
i can the windowa must as was was not in a ward to be a ralling stone

i saw a roadest to she wes
you filled in a mander
in the starling stand
i can the barter in a man was but it is
do you, mister jones?


you that really foul changing
and your keep the say
it is nothing is the light

play "lonnlethout is blowin' in the great can a roor fist ling con ancander man the triend was a black buck that door you feel my love

for the ramemare man of me
it's a road
put ya' look to get
and walking to be done

the answer in the gur
there in the wind

better the sun anyway
and your keep it along for me
in the grown
----- diversity: 0.5
----- Generating with seed: "planes
you that build all the bombs
you "
planes
you that build all the bombs
you metter cater
the man you dead
heard to go for the trind
i clates was spinning down the rage
i was parisen in the gurf

the onever is id off him but shore we know your live on my live a man of on my mind
play it for me
in the grown
i heard the someone they witery that can gett
blake it for the man be a someone
to make and the throme me and the ground
it's the goont to blowing it ain't no use in to 
----- diversity: 1.0
----- Generating with seed: "planes
you t

tome, i can' on blong deach to the wind
where the windows and the brostle mixtearthen something is blowin' it go
him with sty
ridelly see a wast surse”

an, when it gone
play "gleve bot like be love
it's the tomen
that he could-a been
the cops same the side before
it's the partong
 in cost it
i was gones it is
do mad, it's just a menne

----- diversity: 1.0
----- Generating with seed: "s returned into sand
vanished from my ha"
s returned into sand
vanished from my hand

he really for and the jong that you would doun a jamz to use and the good
though this man, play a spelp"
thare arout can get your enes
you gover's fon
s one that call
i that jungle jangle mornick
the didwapping to brise, get just a man, and ithid' on i world
tho ght thiw it up to the bick that it for strick inly bighs and if his came thinks play that goonh whe windly happele to quit
goody was 
10/10 [==============================] - 43s 4s/step - loss: 1.0745 - categorical_crossentropy: 1.0745 - accuracy: 0.6701
Epoch 

In [52]:
def generateLyricLine(seedString, model, lyrics):
    # re-pre processing
    chars = sorted(list(set(lyrics)))
    char_indices = dict((c, i) for i, c in enumerate(chars))
    indices_char = dict((i, c) for i, c in enumerate(chars))

    # finding num. of characters in text & number of unique characters
    n_characters = len(lyrics)
    n_vocabulary = len(chars)

    seqlen = 40
    step = seqlen
    sentences = []
    for i in range(0, len(lyrics) - seqlen - 1, step):
        sentences.append(lyrics[i: i + seqlen + 1])
    x = np.zeros((len(sentences), seqlen, len(chars)), dtype=np.bool)
    y = np.zeros((len(sentences), seqlen, len(chars)), dtype=np.bool)
    for i, sentence in enumerate(sentences):
        for t, (char_in, char_out) in enumerate(zip(sentence[:-1], sentence[1:])):
            x[i, t, char_indices[char_in]] = 1
            y[i, t, char_indices[char_out]] = 1
    def sample(preds, temperature=1.0):
        """Helper function to sample an index from a probability array."""
        preds = np.asarray(preds).astype('float64')
        preds = np.exp(np.log(preds) / temperature)  # softmax
        preds = preds / np.sum(preds)                #
        probas = np.random.multinomial(1, preds, 1)  # sample index
        return np.argmax(probas)  
    
    generated = ''
    output = ''
    sentence = seedString
    sys.stdout.write(generated)
    for i in range(200):
        x_pred = np.zeros((1, 40, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.
        preds = model.predict(x_pred, verbose=0)
        next_index = sample(preds[0, -1], 0.2)
        next_char = indices_char[next_index]
        sentence = sentence[1:] + next_char
        # sys.stdout.write(next_char)
        output = output + next_char
        sys.stdout.flush()
    return output

In [22]:
artist1lyrics = makeLyricsText("Taylor Swift")

Searching for songs by Taylor Swift...

Song 1: "​cardigan"
Song 2: "​exile"
Song 3: "Lover"
Song 4: "​the 1"
Song 5: "Look What You Made Me Do"
Song 6: "​betty"
Song 7: "End Game"
Song 8: "ME!"
Song 9: "You Need To Calm Down"
Song 10: "​august"
Song 11: "...Ready for It?"
Song 12: "Blank Space"
Song 13: "​my tears ricochet"
Song 14: "Delicate"
Song 15: "​invisible string"
Song 16: "Call It What You Want"
"Bad Blood (Remix)" is not valid. Skipping.
Song 17: "​the last great american dynasty"
"I Don’t Wanna Live Forever" is not valid. Skipping.
Song 18: "Cruel Summer"
Song 19: "Style"
Song 20: "​seven"

Reached user-specified song limit (20).
Done. Found 20 songs.
Songs grabbed:20
vintage tee, brand new phone
high heels on cobblestones
when you are young, they assume you know nothing
sequin smile, black lipstick
sensual politics
when you are young, they assume you know nothing

but i knew you
dancin' in your levi's
drunk under a streetlight, i
i knew you
hand under my sweatshirt
baby, k

r
nnnyn
' nnnyyn"yanant mnnnyas"nndn
n"nm
 ntnnnynkn
n"nndnnyyn"ynnyyn"yntnnnyyn
y
sg
n""nlyayas
yyy
ndn
nyneyynnnyntnneyenys"
dnunny"yyn"yn"ynd"
ssd
nn
ttt

nsan'y"y'"n anyiynnnyn
' ensny
dhsynnny"ya

In [29]:
artist2lyrics = makeLyricsText("Bob Dylan")

Searching for songs by Bob Dylan...

Song 1: "Murder Most Foul"
Song 2: "Blowin’ in the Wind"
Song 3: "The Times They Are A-Changin’"
Song 4: "All Along the Watchtower"
Song 5: "Like a Rolling Stone"
Song 6: "Make You Feel My Love"
Song 7: "Hurricane"
Song 8: "Don’t Think Twice, It’s All Right"
Song 9: "Mr. Tambourine Man"
Song 10: "A Hard Rain’s A-Gonna Fall"
Song 11: "Subterranean Homesick Blues"
Song 12: "Desolation Row"
Song 13: "Tangled Up in Blue"
Song 14: "Masters of War"
Song 15: "Knockin’ on Heaven’s Door"
Song 16: "Tempest"
Song 17: "It’s Alright, Ma (I’m Only Bleeding)"
Song 18: "Girl from the North Country"
Song 19: "I Contain Multitudes"
Song 20: "Ballad of a Thin Man"

Reached user-specified song limit (20).
Done. Found 20 songs.
Songs grabbed:20
'twas a dark day in dallas, november '63
a day that will live on in infamy
president kennedy was a-ridin' high
good day to be livin' and a good day to die
being led to the slaughter like a sacrificial lamb
he said, "wait a minute

In [38]:
taylor1 = generateLyricLine("this must be forty characters long betty", artist1model, artist1lyrics)
print(taylor1)


and you'd came

i was true
this i did call
that i dod't like a fay
and you'd crustime
i think i'vers go downthi has d dich
to tell you'd call
i've seen that?
is it cool that's deain't that i to dell 


In [39]:
lastTaylor1 = taylor1[-40:]
print(lastTaylor1)

s it cool that's deain't that i to dell 


In [41]:
bob1 = generateLyricLine(lastTaylor1, artist2model, artist2lyrics)
print(bob1)

my back "now you do the time the wind

you that they to draging and stand in the gut stripp
in is not he leany peeps

they mublething to the pregiled to way
they do rear

the watch that he was was sta


In [42]:
taylor2 = generateLyricLine(bob1[-40:], artist1model, artist1lyrics)
bob2 = generateLyricLine(taylor2[-40:], artist2model, artist2lyrics)
song = taylor1 + bob1 + taylor2 + bob1
print(song)


and you'd came

i was true
this i did call
that i dod't like a fay
and you'd crustime
i think i'vers go downthi has d dich
to tell you'd call
i've seen that?
is it cool that's deain't that i to dell my back "now you do the time the wind

you that they to draging and stand in the gut stripp
in is not he leany peeps

they mublething to the pregiled to way
they do rear

the watch that he was was stale, i'l inded you'd call
anv reading to teent's a that?
is it chill that you'll seevers
the said "james
severy
tantire, lover

see this i mast's that to deaving to teat mest me do
lough

be babyed
be'my back "now you do the time the wind

you that they to draging and stand in the gut stripp
in is not he leany peeps

they mublething to the pregiled to way
they do rear

the watch that he was was sta


In [43]:
migosLyrics = makeLyricsText("Migos")
migosModel = createModel(migosLyrics)

Searching for songs by Migos...

Song 1: "Bad and Boujee"
"Versace (Remix)" is not valid. Skipping.
Song 2: "T-Shirt"
Song 3: "Slippery"
Song 4: "Walk It Talk It"
Song 5: "Stir Fry"
Song 6: "Kelly Price"
Song 7: "Narcos"
Song 8: "Get Right Witcha"
Song 9: "Fight Night"
Song 10: "Handsome and Wealthy"
Song 11: "Notice Me"
Song 12: "Call Casting"
Song 13: "Deadz"
Song 14: "Cocoon"
Song 15: "Hannah Montana"
Song 16: "Freak No More"
Song 17: "BBO (Bad Bitches Only)"
Song 18: "Bando"
Song 19: "China Town"
Song 20: "What the Price"

Reached user-specified song limit (20).
Done. Found 20 songs.
Songs grabbed:20
you know, young rich niggas
you know somethin', we ain't really never had no old money
we got a whole lotta new money though, hah
(if young metro don't trust you, i'm gon' shoot you)
hey

raindrop (drip), drop-top (drop-top)
smokin' on cookie in the hotbox (cookie)
fuckin' on your bitch, she a thot, thot (thot)
cookin' up dope in the crockpot (pot)
we came from nothin' to somethin', ni

Epoch 1/50
15/16 [===========================>..] - ETA: 0s - loss: 3.2598 - categorical_crossentropy: 3.2598 - accuracy: 0.1381
----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "n trouble (niggas in trouble)
you niggas"
n trouble (niggas in trouble)
you niggas   e     o ae oa  i a  loo o o to t o   d  oa a  o  a o o  o i  t aooa  oo  a a   on o o  o  t o a aat h t a a o    o o  t a   oa l t o  t  i oo o t  t a o  t   o io  e t  t a ao      o p t i h on  o  i   o  o t t ta oo t  t  io   t o aaa     t a to o aa  t    i   o  o i t    o   t a  a e  t  t a  t o i  o  t i t t it t t  a  n a le a    o   a  o  o oa     oo     oi a    o   oo ao    o   o tiea  i
----- diversity: 0.5
----- Generating with seed: "n trouble (niggas in trouble)
you niggas"
n trouble (niggas in trouble)
you niggas  t hhoo tt (, noo o raet heo i
i    owe haea  lli loeat eoitoc t tdsoaiahe  i aheto at
wilega koaoi ntid  idaoa( ittokohet  l tain  odute t oa   (one d
anna tetbeee oha

keep a beretta, walk it like i got the bad in the bad ahe walk it like a moke the bitches it like it like it leke treppin' it like it like i malk it the bad it like it all it like i talk it like it (prackeit the bad it like it like it like i me kitches it like it like i calk it like i got it the prick it like it like it like i talk it like it like i gat in the bad ahe (walk it like the watches it like it like it 
----- diversity: 0.5
----- Generating with seed: "do better, never settle
keep a beretta, "
do better, never settle
keep a beretta, it on bitcheo freralliget tall aom the like i pad itche bitches the dopeah, i trop, i get watche site like in me loke lotce (yeahes in yout the moke, pllkein the wout)
rick it like tron the like it lite (prich, ckera serrost ckere bitche got ind treck it loke trappit like i bake a salking like trap i calk it like i bitches it she cut the bapdin' see bady walk it it blike like u fle le
sing the tra
----- diversity: 1.0
----- Generating with seed: "

trapped on my couse (bad bitche fout the price nigga on the bitches on the monga got a pust (go)
bad but i manta got a din' the beah (nigga co co on the couce (she win' with the price (yeah)
me me freak (bad)
up i put  t k it like it with the wiamn take i talk it like i talk it like a ple the gon't niggas on t
----- diversity: 1.0
----- Generating with seed: "tin' the rock like i'm rocsi
she sweatin"
tin' the rock like i'm rocsi
she sweatin't cust tway, cal (it (can' keris)
i pover strras (yeah)
bot like outh the cause (whoh)
fugid, iupundirs
and curle
dove know your)
i dalk it luckeg a pup
i got ad nighas up, ads-me off i'm fomy, ske the mit's bandoendrem pin' of no moret like wave ring ay hith houst (proubie)
 a maknoug i pa dle, yea mige bit
wheth-to tow  (shad i'ma, a coneer a diade

win'in' (zit)
brih s'm (shed)
i git like no b
16/16 [==============================] - 40s 2s/step - loss: 1.7781 - categorical_crossentropy: 1.7781 - accuracy: 0.4882
Epoch 10/50
15/16 [==============

[choppe assted aid bitches a5 that bed)

bat vath)
spockety dut tay
she with thes, phe fles, blad hur, ro frye
if y uh, goh!)

pruces ouh)
way bit)
flloly see (geah)
she asg (woo)
(got)

bracks)
up), fresh o
16/16 [==============================] - 38s 2s/step - loss: 1.4968 - categorical_crossentropy: 1.4968 - accuracy: 0.5677
Epoch 14/50
16/16 [==============================] - ETA: 0s - loss: 1.4462 - categorical_crossentropy: 1.4462 - accuracy: 0.5804
----- Generating text after Epoch: 13
----- diversity: 0.2
----- Generating with seed: "
bad bitches on me, only bad bitches wan"

bad bitches on me, only bad bitches wanna be a freak no more
she she freak no more
i got douse with the bando (bando)
trappin' out the house with the bando (bando)
trapped out the bando (bando)
trapped out the bando (bando)
trapped out the bando (bando)
trapped out the bando (bando)
trapped out the bando (bando)
trapped out the bando (bando)
trapped out the bando (bando)
trapped out the bando (bando)
trapp

...

and you know i got that rol' on my chaice (shite)
we che she freak no more
she don't wanna be a freak no more
she don't wanna be a freak no more
don't wanna be a freak no more
she don't wanna be a freak no more
don't wanna be freakin', no more
she don't wanna be a freak no more
she don't wanna be a freak no more
she don't wanna be a freak no more
don't wanna be a freak no more
don't wanna be a freak no more
i got molly, i got whi
----- diversity: 0.5
----- Generating with seed: "ice me
and you know i got that rol' on m"
ice me
and you know i got that rol' on me
i pull up, i pull up, i pull up, i pull up
i had you choest on (club it), cags of the price (yeah)
had the cloble


i got white
i got mana, she fuck with the right
i'ma knock the pussy out like fight nigga sail she sucking (with you)
don't know why i came in the coupe i gutta (get)
he seen watch me (yeah, yeah, yeah, yeah, yeah, yeah, yeah (trap)
trappin' out the house wit' my fices sone hannah 
----- diversity: 1.0
----- Genera

i talk it like i talk it, walk it, walk it like i talk it (yeah)
walk it, walk it like i talk it (woo)
walk it like i talk it (walk it), walk it like i talk it (walk it), walk it like i talk it (walk it), walk it like i talk it (walk it), walk it like i talk it (woo)
walk it like i talk it (weam 'em)
hold up, come from the deads on the windows (woah)
trapped out the bando (bando)
trappin'
----- diversity: 1.0
----- Generating with seed: "in, yeah
so much money they use rakes (t"
in, yeah
so much money they use rakes (talks)
if i wat theme fink and came, rocks in the mass
watt (polls)
)
get rawht getting theme ic, her rain 4preally
i got what?" (cookee)
they doing bry, i'm someland nigga bout robit (fieg)
lil' and bad, i slide
, jusmen fice ow yeah, yeah

if she sket my my pront
 i wanna be freakin'
doads, trappin', trappin', trappins
my blist (salliis)
i'ma stlat these both my bongeroking lota my top like pomss
16/16 [==============================] - 38s 2s/step - loss: 0.8654 - categ

we got try they familos?
don't that way
takeoff, ranny carnin' for jumindrobat, but it and to met on my chains (uh?!)
you can just do modign wiy, mama told (trap, some andiset, no more
i she douckon a tig flewe, that be tresh out a king (wish)

walk it, wear the door (prro-ceata forgiggage)
seek and you niggas
16/16 [==============================] - 38s 2s/step - loss: 0.8166 - categorical_crossentropy: 0.8166 - accuracy: 0.7570
Epoch 40/50
15/16 [===========================>..] - ETA: 0s - loss: 0.8083 - categorical_crossentropy: 0.8083 - accuracy: 0.7602
----- Generating text after Epoch: 39
----- diversity: 0.2
----- Generating with seed: ")
uh, goyard, havin' them racks and no r"
)
uh, goyard, havin' them racks and no runble
i got molly, i got white
i got molly, i got white
i get your life (hey)
wanna bitch same on me (calo)
ice fire, i'm doin' whota told you
 undeese and same color ter fice of ride (choice of ride, yeah

walk it like i talk it (walk it), walk it like i talk it (w

trappin' out the house wit' the boards on the windows (woah)
trapped out the bando (bando)
trapped out the bando (bando)
trapped out the bando (bando)
trapped out the bando (bando)
trapped out the bando (bando)
trapped out the bando (bando)
trapped out the bando (bando)
trapped out the bando (bando)
trapped out the bando (bando)
trapped out the bando (bando)
trapped out the bando (bando)
trapped out the bando (bando)
trapped
----- diversity: 0.5
----- Generating with seed: "ndo (bando)
trappin' out the house wit' "
ndo (bando)
trappin' out the house wit' my fifess (i'm not like the sack (she)
i'ma cela molly, i got white
i got molly, you niggas in trouble (
i money auntie (choperide)
niggas starvin', yeah (woo)
bad bitches only, bad bitches on me (woo)
bad bitches only, bad bitches only, bad bitches on me (woo, woo, woo)

yeah, trap, trappin', trappin', trappin' all damn night
hit i wanna (lite to ready, bricks on me (whoo)
thank my chain (but)
sh
----- diversity: 1.0
----- Generating 

uh, erays in trouble, you niggas in trouble
you niggas in trouble
you niggas in trouble
you niggas in trouble
you niggas in trouble
you niggas in trouble
you niggas in trouble, you niggas in trouble (
hoh)
you niggas in trouble (
hoh, yeah, yeah, yeah)
(magnífico)
this it?
bolliezzely, bad bitches only bad bitches only, bad bitches on me (bad)
bad bitches only, 
----- diversity: 1.0
----- Generating with seed: " up in the mornin' (nah)
bad bitches onl"
 up in the mornin' (nah)
bad bitches only, said she need to choppin' up dop, woldy goom bitches only, bad bitches on me (holdsh)
shoters gon a tord (narco)
juhed rough like andors

white i'm alk it (walk it), walk it like i talk it, walk it, walk it, walk it like i talk it (woo)
walk it like i talk it (let's go)
watchin' (ask)
i'ma gut rolin' list (thow y'm same cash rock they white i meah she dun'in' bitch, wheres look ainta out chingr
16/16 [==============================] - 38s 2s/step - loss: 0.7404 - categorical_crossentropy: 0.7404

In [ ]:
def createSongCollab(artist1, artist2):
    artist1fullLyrics = makeLyricsText(artist1)
    artist2fullLyrics = makeLyricsText(artist2)
    artist1model = createModel(artist1)
    artist2model = createModel(artist2)
    artist1line1 = generateLyricLine(artist2fullLyrics[0:40], artist1, artist1fullLyrics)
    artist2line1 = generateLyricLine(artist1line1[-40:], artist2, artist2fullLyrics)
    artist1line2 = generateLyricLine(artist2line1[-40:], artist1, artist1fullLyrics)
    artist2line2 = generateLyricLine(artist1line2[-40:], artist2, artist2fullLyrics)
    song = artist1line1 + artist2line1 + artist1line2 + artist2line2
    return song

print(createSongCollab("Taylor Swift", "Bob Dylan"))

In [44]:
print(taylor1)


and you'd came

i was true
this i did call
that i dod't like a fay
and you'd crustime
i think i'vers go downthi has d dich
to tell you'd call
i've seen that?
is it cool that's deain't that i to dell 


In [45]:
print(taylor2)

le, i'l inded you'd call
anv reading to teent's a that?
is it chill that you'll seevers
the said "james
severy
tantire, lover

see this i mast's that to deaving to teat mest me do
lough

be babyed
be'


In [55]:
artist1line1 = generateLyricLine("this must be forty characters long betty", artist2model, artist2lyrics)
artist2line1 = generateLyricLine(artist1line1[-40:], migosModel, migosLyrics)
artist1line2 = generateLyricLine(artist2line1[-40:], artist2model, artist2lyrics)
artist2line2 = generateLyricLine(artist1line2[-40:], migosModel, migosLyrics)
song = artist1line1 + artist2line1 + artist1line2 + artist2line2
print(song)

KeyError: '('

In [ ]:
""